# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [31]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,54.18,65,80,12.88,AU,2023-10-29 15:23:06
1,1,thompson,55.7435,-97.8558,26.76,63,75,11.50,CA,2023-10-29 15:23:07
2,2,grytviken,-54.2811,-36.5092,41.67,95,100,7.18,GS,2023-10-29 15:23:07
3,3,mingshui,36.7167,117.5000,63.28,47,55,10.83,CN,2023-10-29 15:23:07
4,4,isafjordur,66.0755,-23.1240,34.02,80,36,3.40,IS,2023-10-29 15:23:08


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City',
    alpha = 0.75
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_df = city_data_df
filtered_city_df = filtered_city_df.loc[(filtered_city_df['Max Temp'] >= 65) & (filtered_city_df['Max Temp'] <= 85)] 
filtered_city_df = filtered_city_df.loc[(filtered_city_df['Wind Speed'] < 15)]
filtered_city_df = filtered_city_df.loc[(filtered_city_df['Cloudiness'] < 20)]


# Drop any rows with null values
filtered_city_df = filtered_city_df.dropna()

# Display sample data
filtered_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,guerrero negro,27.9769,-114.0611,75.87,47,0,8.77,MX,2023-10-29 15:23:08
10,10,uturoa,-16.7333,-151.4333,78.89,80,15,14.67,PF,2023-10-29 15:23:09
19,19,dabwali,29.9500,74.7333,72.39,32,2,5.86,IN,2023-10-29 15:23:11
24,24,merizo village,13.2658,144.6691,82.13,94,8,8.19,GU,2023-10-29 15:23:12
25,25,kani keli,-12.9536,45.1033,79.86,84,2,5.66,YT,2023-10-29 15:23:12
...,...,...,...,...,...,...,...,...,...,...
545,545,shahritus,37.2637,68.1385,71.02,30,9,3.36,TJ,2023-10-29 15:25:18
555,555,tabou,4.4230,-7.3528,79.03,85,1,5.30,CI,2023-10-29 15:25:22
560,560,cannonvale,-20.2833,148.7000,72.79,67,17,7.70,AU,2023-10-29 15:25:23
566,566,isangel,-19.5500,169.2667,73.72,65,14,9.86,VU,2023-10-29 15:25:25


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(filtered_city_df, columns = ['City', 'Country', 'Lat', 'Lng', 'Humidity'])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''


# Display sample data
hotel_df.sample(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
79,al wajh,SA,26.2455,36.4525,41,
209,newman,US,37.3138,-121.0208,29,
566,isangel,VU,-19.5500,169.2667,65,
132,hafar al-batin,SA,28.4328,45.9708,31,
385,chemin grenier,MU,-20.4872,57.4656,81,
25,kani keli,YT,-12.9536,45.1033,84,
376,qaisumah,SA,28.3112,46.1273,31,
7,guerrero negro,MX,27.9769,-114.0611,47,
372,pacific grove,US,36.6177,-121.9166,20,
300,fortuna,US,40.5982,-124.1573,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [1]:
"""
# example near location w/ radium filter url from geoapify documentation:
# https://api.geoapify.com/v2/places?categories=public_transport.subway&filter=circle:-0.07071648508463113,51.50848194136378,1000&bias=proximity:-0.07071648508463113,51.50848194136378&limit=20&apiKey=YOUR_API_KEY
lat = 36.6177
lng = -121.9166
radius = 10000
requests.get(f
'https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:{lng},{lat},{radius}&bias=proximity:{lng},{lat}&limit=20&apiKey={geoapify_key}').json()
"""

"\n# example near location w/ radium filter url from geoapify documentation:\n# https://api.geoapify.com/v2/places?categories=public_transport.subway&filter=circle:-0.07071648508463113,51.50848194136378,1000&bias=proximity:-0.07071648508463113,51.50848194136378&limit=20&apiKey=YOUR_API_KEY\nlat = 36.6177\nlng = -121.9166\nradius = 10000\nrequests.get(f\n'https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:{lng},{lat},{radius}&bias=proximity:{lng},{lat}&limit=20&apiKey={geoapify_key}').json()\n"

In [29]:
# Set parameters to search for a hotel
categories = 'accommodation.hotel'
radius = 10000

params = {
    'categories':categories,
    'apikey':geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionary and convert to JSON format
    name_address = requests.get(f'https://api.geoapify.com/v2/places?categories={categories}&filter=circle:{longitude},{latitude},{radius}&bias=proximity:{longitude},{latitude}&limit=20&apiKey={geoapify_key}').json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
guerrero negro - nearest hotel: Plaza sal paraiso
uturoa - nearest hotel: Hawaiki Nui hotel
dabwali - nearest hotel: No hotel found
merizo village - nearest hotel: No hotel found
kani keli - nearest hotel: Hôtel Le Jardin Maoré
avarua - nearest hotel: Paradise Inn
tolanaro - nearest hotel: Hôtel Mahavokey
grand baie - nearest hotel: Safari
az zuwaytinah - nearest hotel: No hotel found
murzuq - nearest hotel: فندق باريس - مرزق
ceres - nearest hotel: Howard Johnson by Wyndham Modesto Ceres
al wajh - nearest hotel: فندق رالين 2
adrar - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
koungou - nearest hotel: Hôtel Trévani
sahawar - nearest hotel: No hotel found
antalaha - nearest hotel: Hôtel Florida
grand-bassam - nearest hotel: Hôtel 3G
hafar al-batin - nearest hotel: فندق الفرحان
kill devil hills - nearest hotel: Mariner Days Inn & Suites
coquimbo - nearest hotel: Hotel Iberia
arraial do cabo - nearest hotel: No hotel found
san antonio de pale

,City,Country,Lat,Lng,Humidity,Hotel Name
7,guerrero negro,MX,27.9769,-114.0611,47,Plaza sal paraiso
10,uturoa,PF,-16.7333,-151.4333,80,Hawaiki Nui hotel
19,dabwali,IN,29.9500,74.7333,32,No hotel found
24,merizo village,GU,13.2658,144.6691,94,No hotel found
25,kani keli,YT,-12.9536,45.1033,84,Hôtel Le Jardin Maoré
...,...,...,...,...,...,...
545,shahritus,TJ,37.2637,68.1385,30,Marhabo Hotel
555,tabou,CI,4.4230,-7.3528,85,hôtel le rochet
560,cannonvale,AU,-20.2833,148.7000,67,Whitsundays Rainforest Retreat
566,isangel,VU,-19.5500,169.2667,65,Tanna Lodge


In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'


params = {
    'categories':categories,
    'apikey':geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary and convert to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City',
    alpha = 0.75
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)